# CLEAR

Chargemaster Location-based Exploration for Affordability & Reform

This is the notebook file primarily responsible for pre-processing data, attaching important information, and generating database files for the github page. Below you can find all information about how data is processed from the downloaded `.csv` files found on most hospital sites. This is an exploratory project focused on creating interactive visualzations and tools to better inform people about their healthcare. The repo can always be maintained by downloading the most current year data for the specific hospital and putting it through the scripts. It should be noted that this is NOT a comprehensive list, but it can potentially be scaled to a full working-standalone site with enough time. 

All pre-processing code is written in python. See the `.html` files for how the D3 visualizations work. 

## How it works (Copied from README)

Hospitals that have been added to this 'web-app' are stored in a `.csv` file for quick look up and ease of access. This points to the loc of it's Charge Master `.json` file which is then queried for the specific procedure. Hospitals are gathered from the CSV list based on a radius look-up provided by the user. If a hospital in the radius does not offer the service, it will not display the price point compared to others in the radius. 

Currently limited to 500 procedures due to file size limits and me not wanting to set up a server/database for this. Parquet only works server side so i can't do iterative testing before publishing to pages, and pages deployments can take a while. Will consider moving to parquet system after front-end is stable and working as envisioned.

## List of Hospitals

These are the hospital's which data has been gathered and processed for thus far:

| State    | Hospital Name                     | Zipcode     | Date                 | File Size    | Link                                                            |
|----------|--------------------------------|-------------|-------------------|-------------|------------------------------------------------|
| NC        | Duke University Hospital     |     27710    |      09/2025      |   3.32 GB   |    [Link](https://www.dukehealth.org/paying-for-care/what-duke-charges-services) |
| NC        | AdventHealth (Hendersonville)   |     28792   |   09/2025    |      1.48 GB           |    |
| NC | UNC Medical Center | 27514 | 09/2025 | 201 MB | [Link](https://rca.centaurihs.com/ptapp/#d4ccc071fab9c79f17e52dc5b243ef668affc5e569aafa907c5b4c81f0a89284) |
| NC | UNC Rex Hospital | 27606 | 09/2025 | 121 MB | [Link](https://www.unchealth.org/records-insurance/standard-charges) |
| NC | WakeMed North Hospital | 27614 | 09/2025 | 56.1 MB | [Link](https://www.wakemed.org/sites/default/files/PricingTransparency/566017737_wakemed-raleigh-campus-and-north-hospital_standardcharges.csv) |
| SC        | MUSC Health-University Medical Center (Charleston) |   29425   | 09/2025 | 11.8 MB |  [Link](https://muschealth.org/patients-visitors/billing/price-transparency) |
| VA | Inova Fairfax Hospital (Falls Church) | 22042 | 09/2025 | 11.3 MB | [Link](https://www.inova.org/patient-and-visitor-information/hospital-charges) |

#### Top Hospitals in Every State 

A list of hospitals that should be added at a later date.

- Alaska: Providence Alaska Medical Center (Anchorage) and Fairbanks Memorial Hospital
- Alabama: University of Alabama at Birmingham Hospital        
- Arizona: Mayo Clinic-Phoenix        
- Arkansas: Washington Regional Medical Center (Fayetteville)        
- California: Cedars-Sinai Medical Center (Los Angeles), UCLA Medical Center (Los Angeles), Stanford Health Care-Stanford Hospital (Palo Alto), UC San Diego Health-LaJolla and Hillcrest Hospitals, and UCSF Health-UCSF Medical Center (San Francisco)
- Colorado: UCHealth University of Colorado Hospital (Aurora)        
- Connecticut: Yale New Haven Hospital        
- Delaware: ChristianaCare Hospitals (Newark)        
- Florida: Mayo Clinic-Jacksonville        
- Georgia: Emory University Hospital (Atlanta)        
- Hawaii: Queen’s Medical Center (Honolulu)        
- Idaho: St. Luke’s Regional Medical Center (Boise)        
- Illinois: Northwestern Medicine-Northwestern Memorial Hospital (Chicago) and Rush University Hospital (Chicago)         
- Indiana: Indiana University Health Medical Center (Indianapolis)        
- Iowa: University of Iowa Hospitals and Clinics (Iowa City)        
- Kansas: University of Kansas Hospital (Kansas City)        
- Kentucky: University of Kentucky Albert B. Chandler Hospital (Lexington) 
- Louisiana: Ochsner Medical Center (New Orleans)        
- Maine: Maine Medical Center (Portland)        
- Maryland: Johns Hopkins Hospital (Baltimore)        
- Massachusetts: Massachusetts General Hospital (Boston) and Brigham and Women’s Hospital (Boston)       
- Michigan: University of Michigan Health-Ann Arbor        
- Minnesota: Mayo Clinic (Rochester)        
- Mississippi: Mississippi Baptist Medical Center (Jackson)        
- Missouri: Barnes-Jewish Hospital (St. Louis)        
- Montana: Billings Clinic        
- Nebraska: Nebraska Medicine-Nebraska Medical Center (Omaha)        
- Nevada: Renown Regional Medical Center (Reno)        
- New Hampshire: Dartmouth Hitchcock Medical Center (Lebanon)        
- New Jersey: Hackensack University Medical Center at Hackensack University Health      
- New Mexico: Presbyterian Hospital (Albuquerque)        
- New York: NYU Langone Hospitals (New York City), New York-Presbyterian Hospital-Columbia and Cornell (New York City), Mount Sinai Hospital (New York City), and North - Shore University Hospital at Northwell Health (Manhasset)        
- ~~North Carolina: Duke University Hospital (Durham)~~       
- North Dakota: Sanford Medical Center Fargo        
- Ohio: Cleveland Clinic        
- Oklahoma: St. Francis Hospital-Tulsa        
- Oregon: OHSU Hospital (Portland)        
- Pennsylvania: Hospitals of the University of Pennsylvania-Penn Presbyterian (Philadelphia)        
- Rhode Island: Miriam Hospital (Providence)        
- ~~South Carolina: MUSC Health-University Medical Center (Charleston)~~        
- South Dakota: Sanford USD Medical Center (Sioux Falls)        
- Tennessee: Vanderbilt University Medical Center (Nashville)        
- Texas: Houston Methodist Hospital and UT Southwestern Medical Center (Dallas)
- Utah: University of Utah Hospital (Salt Lake City)        
- Vermont: University of Vermont Medical Center (Burlington)        
- ~~Virginia: Inova Fairfax Hospital (Falls Church)~~        
- Washington: UW Medicine-University of Washington Medical Center (Seattle)        
- West Virginia: West Virginia University Hospitals (Morgantown)        
- Wisconsin: UW Health University Hospital (Madison)

As by Becker https://www.beckershospitalreview.com/rankings-and-ratings/us-news-top-hospitals-by-state-for-2023-24/

## Outside Sources Used

- zip_centroids.csv courtesy of SimpleMaps data https://simplemaps.com/data/us-zips.
- CMS.gov data 
    - for top 200 HCPCS and CPT codes billed for 2024 & top 100 lab codes. [Link](https://www.cms.gov/data-research/statistics-trends-and-reports/medicare-fee-for-service-parts-a-b/medicare-utilization-part-b)
    - PFALL25 
    - PFS (Physician scheduling fee) for mapping HCPCS/CPT codes to medicare rates [Link](https://www.cms.gov/medicare/payment/fee-schedules/physician/national-payment-amount-file)
    - ASC Rates for mapping HCPCS/CPT codes to ambulatory rates [Link](https://www.cms.gov/medicare/payment/prospective-payment-systems/ambulatory-surgical-center-asc/asc-payment-rates-addenda)




***

## MAJOR CHANGES

- Moved all HCPCS/CPT top 200 & lab codes into a single file
- removed depreciated codes from 2024 to 2025 since we're working with 2025 CMs
- pulled CPT code descriptions from CMS RVU data for early 2025 (jan)
- validated 2024 codes against 2025 to ensure all are being caught properly
- update bundler to reflect this

data processing for hospitals will need to be updated to reflect these changes

## Data Processing

CSV files are too large to store on github, thus they are downloaded locally, converted to the necessary format, then uploaded. If you want to perform conversions yourself you will need to find the specific hospital chargemaster and document in the notebook accordingly.

Not all Charge Masters (CM) are formatted the same, as such, to keep this notebook from growing too large, custom python scripts will be made for unique CM's. This matters beccause some hospitals are regional or statewide 'chains' but can vary prices between locations. For example, 

**AdventHealth**
- AdventHealth Orlando
- AdventHealth Tampa
- AdventHealth Hendersonville

all are AdventHealth hospitals, but their prices and available procedures vary per location. However, the same script to clean and process their CM's works because the file structure doesn't change from loc to loc. Normally CM structure only changes from hospital to hospital (brand-wise), but I haven't looked at the majority of US hospitals so this statement might need to be amended. 

Think of this file as more of a "**Controller**" for the cleaning, while the cleaning process is performed by imported functions. Subsections from here on are labeled by State, be sure to check which Hospitals are in each subsection before uploading data. 



***
## Payer & Plan Names

Naming conventions for payer/plans differ across hospitals, making this a pain. Like is an exhaustive regex section to hopefully simplify this so that the functionality of the .html page remains. 

Idk where this fits in, I'll add it later to documentation.

use this to create a comprehensive list of all current payer names.

***
## North Carolina Hospitals

In [2]:

# ======================================================================
# --------------- DUKE HOSPITAL TESTING ----------------
# ======================================================================

import os
import pandas as pd
from scripts.cleaners import transform_wide_to_long_format
from scripts.helpers import add_hospital_entry
from scripts.code_matcher import get_matches
from scripts.cleaners import standardize_payer_name

workspace_root = os.path.dirname(os.path.abspath('CLEAR.ipynb'))
csv_folder = os.path.join(workspace_root, '..', 'ChargeMaster_Project', 'csv_files')
csv_folder = os.path.abspath(csv_folder)

new_hos_dict = {
    'hospital_name': 'Duke University Hospital',
    'city_name': 'Durham',
    'state_name': 'NC',
    'address': '2301 Erwin Road',
    'zip_code': '27110',
}
# add hospital info for Duke University Hospital to hospitals.csv if it doesn't already exist
hospitals_df = add_hospital_entry(new_hos_dict)

# now lets grab all the paths
duke_json_path = hospitals_df[hospitals_df['name'] == new_hos_dict['hospital_name']]['json_path'].values[0]
duke_csv_path = os.path.join(csv_folder, 'DUKE_UNIVERSITY_HOSPITAL_CM.csv')

# load Duke University Hospital csv
duke_df = pd.read_csv(duke_csv_path)

# Duke University Hospital CM Structure is in long format, so we can skip the wide to long transformation step
# duke_long_df = transform_wide_to_long_format(duke_df, verbose=True)

# Now grab matches using the new get_matches function
all_matches = get_matches(duke_df, verbose=True)

# Now apply the payer standardization
all_matches['payer_name'] = all_matches['payer_name'].apply(standardize_payer_name)

# Now we can save to json
all_matches.to_json(duke_json_path, orient='records', lines=True)

# drop file/df from memory to save space
del duke_df
# del duke_long_df
del all_matches
del duke_csv_path
del duke_json_path
del new_hos_dict
del hospitals_df

print("Duke University Hospital test processing complete.")
# ======================================================================


C:\Users\jcing\AppData\Local\Temp\ipykernel_31708\3778326329.py:31: DtypeWarning: Columns (7,8,9,12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  duke_df = pd.read_csv(duke_csv_path)


Loaded the following code sets:
HCPCS Codes: 200 entries
Lab Codes: 100 entries
CPT Codes: 200 entries
All Codes: 469 entries
Identified code columns: ['code|1', 'code|2', 'code|3', 'code|4']
Renamed column 'code|1' to 'code_1'.
Renamed column 'code|1|type' to 'code_1_type'.
Renamed column 'code|2' to 'code_2'.
Renamed column 'code|2|type' to 'code_2_type'.
Renamed column 'code|3' to 'code_3'.
Renamed column 'code|3|type' to 'code_3_type'.
Renamed column 'code|4' to 'code_4'.
Renamed column 'code|4|type' to 'code_4_type'.
Column 'code_1': Found 0 HCPCS matches, 0 Lab matches, 0 CPT matches.
No matches found in column 'code_1'.
Column 'code_2': Found 76966 HCPCS matches, 19987 Lab matches, 0 CPT matches.
Reassigning code_2 to 'code' and 'code_2_type' to 'type' for non-empty matches DataFrames.
Column 'code_2': Combined matches shape: (96953, 27)
Column 'code_2': Unique matches after dropping duplicates: 28057
Column 'code_3': Found 0 HCPCS matches, 0 Lab matches, 772 CPT matches.
Reassi

In [ ]:
# ======================================================================
# --------------- ADVENTHEALTH HOSPITAL  ----------------
# ======================================================================

# Load AdventHealth Hendersonville, NC paths
hos_name = 'AdventHealth'
city_name = 'Hendersonville'
state_name = 'NC'
matching_hospitals = hospitals_df[
    (hospitals_df['name'] == hos_name) &
    (hospitals_df['state'] == state_name) &
    (hospitals_df['city'] == city_name)
]
if not matching_hospitals.empty:
    adv_nc = matching_hospitals.iloc[0]
else:
    print(f"Hospital '{hos_name}' not found in the dataset")
    adv_nc = None

# grab json path for AdventHealth Hendersonville, NC
adv_nc_json_path = adv_nc['json_path']

# load a single csv file from csv_folder for testing
adv_nc_csv_path = os.path.join(csv_folder, 'AdventHealth_Hendersonville_CM.csv')

# load AdventHealth Hendersonville, NC csv
adv_nc_df = pd.read_csv(adv_nc_csv_path)

# AdventHealth CM Structure
# ['description', 'drug_information', 'code', 'type',
#    'standard_charge_min', 'standard_charge_max', 'gross_charge',
#    'discounted_cash', 'setting', 'payer_name', 'plan_name',
#    'standard_charge_dollar', 'standard_charge_percentage',
#    'estimated_amount', 'methodology', 'standard_charge_algorithm',
#    'Hospital', 'City', 'State', 'Address']

# Check matches for code column against hcpcs_codes, cpt_codes, and lab_codes
# Output: 
# HCPCS matches: 20845
#   CPT matches: 2884
#   Lab matches: 13555

# Lets drop unneeded columns, and rename some before grabing the matches and saving to json
# NOTE: common naming convention needs to be added before renaming cols
cols_to_drop = ['methodology', 'drug_information', 'standard_charge_algorithm', 'Hospital', 'City', 'State', 'Address']
adv_nc_df = adv_nc_df.drop(columns=cols_to_drop)

# now grab matches
hcpcs_matches = adv_nc_df[adv_nc_df['code'].isin(hcpcs_codes['HCPCS Code'])]
cpt_matches = adv_nc_df[adv_nc_df['code'].isin(cpt_codes['HCPCS Code'])]
lab_matches = adv_nc_df[adv_nc_df['code'].isin(lab_codes['HCPCS Code'])]

# Combine all matches into one dataframe, drop duplicates
match_dfs = [df for df in [hcpcs_matches, cpt_matches, lab_matches] if not df.empty]

if match_dfs:
    all_matches = pd.concat(match_dfs, ignore_index=True).drop_duplicates()
    all_matches['payer_name'] = all_matches['payer_name'].apply(standardize_payer_name)

    # redrop possible duplicates
    all_matches = all_matches.drop_duplicates()

else:
    # Create empty DataFrame with same structure as duke_df if no matches
    all_matches = pd.DataFrame(columns=duke_df.columns)

# There are some duplicate issues, mainly rows where no est. price are given, so lets remove enteries that don't have est. prices
all_matches = all_matches[all_matches['estimated_amount'].notna() & (all_matches['estimated_amount'] != '')]

# Save output data to json file for AdventHealth Hendersonville, NC json path
all_matches.to_json(adv_nc_json_path, orient='records', lines=True)

# drop file/df from memory to save space
del adv_nc_df
del adv_nc
del adv_nc_csv_path
del all_matches

print("AdventHealth Hendersonville, NC test processing complete.")
# ======================================================================

C:\Users\jcing\AppData\Local\Temp\ipykernel_5708\1743173882.py:27: DtypeWarning: Columns (1,2,15) have mixed types. Specify dtype option on import or set low_memory=False.
  adv_nc_df = pd.read_csv(adv_nc_csv_path)


AdventHealth Hendersonville, NC test processing complete.


In [1]:
# ======================================================================
# ---------------  UNC MEDICAL CENTER  ----------------
# ======================================================================

# New added helper functions should reduce code bloat and need to rerun the initial cells
import os
import pandas as pd
from scripts.cleaners import transform_wide_to_long_format
from scripts.helpers import add_hospital_entry
from scripts.code_matcher import get_matches
from scripts.cleaners import standardize_payer_name

workspace_root = os.path.dirname(os.path.abspath('CLEAR.ipynb'))
csv_folder = os.path.join(workspace_root, '..', 'ChargeMaster_Project', 'csv_files')
csv_folder = os.path.abspath(csv_folder)

new_hos_dict = {
    'hospital_name': 'UNC Medical Center',
    'city_name': 'Chapel Hill',
    'state_name': 'NC',
    'address': '101 Manning Dr',
    'zip_code': '27514',
}
# add hospital info for UNC Medical Center to hospitals.csv if it doesn't already exist
hospitals_df = add_hospital_entry(new_hos_dict)

# now lets grab all the paths
unc_json_path = hospitals_df[hospitals_df['name'] == new_hos_dict['hospital_name']]['json_path'].values[0]
unc_csv_path = os.path.join(csv_folder, 'UNC_MEDICAL_CENTER_CM.csv')

# load UNC Medical Center csv
unc_df = pd.read_csv(unc_csv_path)

# UNC Medical Center CM Structure is in wide format, need to transform to long format first
unc_long_df = transform_wide_to_long_format(unc_df, verbose=True)

# Now grab matches using the new get_matches function
all_matches = get_matches(unc_long_df, verbose=True)

# Now apply the payer standardization
all_matches['payer_name'] = all_matches['payer_name'].apply(standardize_payer_name)

# Now we can save to json
all_matches.to_json(unc_json_path, orient='records', lines=True)

# drop file/df from memory to save space
del unc_df
del unc_long_df
del all_matches
del unc_csv_path
del unc_json_path
del new_hos_dict
del hospitals_df

print("UNC Medical Center test processing complete.")


C:\Users\jcing\AppData\Local\Temp\ipykernel_16680\2145981687.py:32: DtypeWarning: Columns (5,6,10,11,19,22,49,52,55,58,61,64,67,70,79,82,91,94,97,100,115,118,127,130,133,136,139,142,145,148,151,154,157,160,163,166,169,172,175,178,181,184,187,190,194,196,199,202,217,220,223,226) have mixed types. Specify dtype option on import or set low_memory=False.
  unc_df = pd.read_csv(unc_csv_path)


Original DataFrame shape: 155232 rows × 239 columns
Available base columns: 17 out of 20 possible
Found 155 payer-specific columns
Found 31 unique payer/plan combinations
Columns being dropped: 211
Expected columns in result: 28
Final DataFrame shape: 1163866 rows × 24 columns
Row multiplication factor: 7.50x (from 155232 to 1163866)
Actual columns dropped: 215
Data efficiency: 0.75 (final data points / original data points)
Original total data points: 37,100,448
Final total data points: 27,932,784
Loaded the following code sets:
HCPCS Codes: 200 entries
Lab Codes: 100 entries
CPT Codes: 200 entries
All Codes: 469 entries
Identified code columns: ['code_1', 'code_2', 'code_3']
Column 'code_1': Found 0 HCPCS matches, 0 Lab matches, 264 CPT matches.
Reassigning code_1 to 'code' and 'code_1_type' to 'type' for non-empty matches DataFrames.
Column 'code_1': Combined matches shape: (264, 26)
Column 'code_1': Unique matches after dropping duplicates: 264
Column 'code_2': Found 7142 HCPCS mat

In [5]:
# ======================================================================
# ---------------  UNC REX HOSPITAL  ----------------
# ======================================================================
# Lets start adding info to the CSV in python rather than manual edits each time
# Load UNC Rex Hospital in Raleigh, NC paths
import os
import pandas as pd
from scripts.cleaners import transform_wide_to_long_format
from scripts.helpers import add_hospital_entry
from scripts.code_matcher import get_matches
from scripts.cleaners import standardize_payer_name

workspace_root = os.path.dirname(os.path.abspath('CLEAR.ipynb'))
csv_folder = os.path.join(workspace_root, '..', 'ChargeMaster_Project', 'csv_files')
csv_folder = os.path.abspath(csv_folder)

new_hos_dict = {
    'hospital_name': 'UNC Rex Hospital',
    'address': '4420 Lake Boone Trail',
    'city_name': 'Raleigh',
    'state_name' : 'NC',
    'zip_code' : '27607'
}

# add hospital info for UNC Medical Center to hospitals.csv if it doesn't already exist
hospitals_df = add_hospital_entry(new_hos_dict)

# now lets grab all the paths
unc_rex_json_path = hospitals_df[hospitals_df['name'] == new_hos_dict['hospital_name']]['json_path'].values[0]
unc_rex_csv_path = os.path.join(csv_folder, 'UNCREX_CM.csv')

# load UNC Rex Hospital csv
unc_rex_df = pd.read_csv(unc_rex_csv_path)

# UNC Rex Hospital CM Structure is in wide format, need to transform to long format first
unc_rex_long_df = transform_wide_to_long_format(unc_rex_df, verbose=True)

# Now grab matches using the new get_matches function
all_matches = get_matches(unc_rex_long_df, verbose=True)

# Now apply the payer standardization
all_matches['payer_name'] = all_matches['payer_name'].apply(standardize_payer_name)

# Now we can save to json
all_matches.to_json(unc_rex_json_path, orient='records', lines=True)

# drop file/df from memory to save space
del unc_rex_df
del unc_rex_long_df
del all_matches
del unc_rex_csv_path
del unc_rex_json_path
del new_hos_dict
del hospitals_df

print("UNC Rex Hospital test processing complete.")

C:\Users\jcing\AppData\Local\Temp\ipykernel_16680\3681096947.py:33: DtypeWarning: Columns (5,6,10,11,19,22,43,46,49,52,55,58,61,64,73,76,88,91,94,109,112,121,124,127,130,133,136,139,142,145,148,151,154,157,160,163,166,169,172,175,178,181,184,187,190,199,202) have mixed types. Specify dtype option on import or set low_memory=False.
  unc_rex_df = pd.read_csv(unc_rex_csv_path)


Original DataFrame shape: 160860 rows × 215 columns
Available base columns: 17 out of 20 possible
Found 125 payer-specific columns
Found 25 unique payer/plan combinations
Columns being dropped: 187
Expected columns in result: 28
Final DataFrame shape: 1057313 rows × 24 columns
Row multiplication factor: 6.57x (from 160860 to 1057313)
Actual columns dropped: 191
Data efficiency: 0.73 (final data points / original data points)
Original total data points: 34,584,900
Final total data points: 25,375,512
Loaded the following code sets:
HCPCS Codes: 200 entries
Lab Codes: 100 entries
CPT Codes: 200 entries
All Codes: 469 entries
Identified code columns: ['code_1', 'code_2', 'code_3']
Column 'code_1': Found 0 HCPCS matches, 0 Lab matches, 191 CPT matches.
Reassigning code_1 to 'code' and 'code_1_type' to 'type' for non-empty matches DataFrames.
Column 'code_1': Combined matches shape: (191, 26)
Column 'code_1': Unique matches after dropping duplicates: 191
Column 'code_2': Found 4222 HCPCS mat

In [5]:
# ======================================================================
# ---------------  WAKE MED NORTH HOSPITAL  ----------------
# ======================================================================

# New added helper functions should reduce code bloat and need to rerun the initial cells
import os
import pandas as pd
from scripts.cleaners import transform_wide_to_long_format
from scripts.helpers import add_hospital_entry
from scripts.code_matcher import get_matches
from scripts.cleaners import standardize_payer_name

workspace_root = os.path.dirname(os.path.abspath('CLEAR.ipynb'))
csv_folder = os.path.join(workspace_root, '..', 'ChargeMaster_Project', 'csv_files')
csv_folder = os.path.abspath(csv_folder)

new_hos_dict = {
    'hospital_name': 'WakeMed North Hospital',
    'city_name': 'Raleigh',
    'state_name': 'NC',
    'address': '10000 Falls of Neuse Rd',
    'zip_code': '27614',
}
# add hospital info for WakeMed North Hospital to hospitals.csv if it doesn't already exist
hospitals_df = add_hospital_entry(new_hos_dict)

# now lets grab all the paths
wake_med_json_path = hospitals_df[hospitals_df['name'] == new_hos_dict['hospital_name']]['json_path'].values[0]
wake_med_csv_path = os.path.join(csv_folder, 'WAKEMED_NORTH_CM.csv')

# load WakeMed North Hospital csv
wake_med_df = pd.read_csv(wake_med_csv_path)

# WakeMed North CM Structure is in wide format, need to transform to long format first
wake_med_long_df = transform_wide_to_long_format(wake_med_df, verbose=True)

# Now grab matches using the new get_matches function
# NOTE: WakeMed North has next to no estimated_amount data
all_matches = get_matches(wake_med_long_df, verbose=True)

# Now apply the payer standardization
all_matches['payer_name'] = all_matches['payer_name'].apply(standardize_payer_name)

# Now we can save to json
all_matches.to_json(wake_med_json_path, orient='records', lines=True)

# drop file/df from memory to save space
del wake_med_df
del wake_med_long_df
del all_matches
del wake_med_csv_path
del wake_med_json_path
del new_hos_dict
del hospitals_df

print("WakeMed North Hospital test processing complete.")

C:\Users\jcing\AppData\Local\Temp\ipykernel_15176\1577898819.py:32: DtypeWarning: Columns (1,3,4,6,7,8,12,13,18,23,24,29,30,35,36,41,47,48,53,54,59,65,66,71,72,77,78,83,84,89,90,95,96,101,102,107,108,113,114,119,120,125,131,132,137,138,143,149,150,155,161,167,173,179,180,185,186,191,192,197,198,203,204,209,210) have mixed types. Specify dtype option on import or set low_memory=False.
  wake_med_df = pd.read_csv(wake_med_csv_path)


Original DataFrame shape: 104112 rows × 211 columns
Available base columns: 19 out of 20 possible
Found 160 payer-specific columns
Found 32 unique payer/plan combinations
Columns being dropped: 181
Expected columns in result: 30
Final DataFrame shape: 681680 rows × 26 columns
Row multiplication factor: 6.55x (from 104112 to 681680)
Actual columns dropped: 185
Data efficiency: 0.81 (final data points / original data points)
Original total data points: 21,967,632
Final total data points: 17,723,680
Loaded the following code sets:
HCPCS Codes: 200 entries
Lab Codes: 100 entries
CPT Codes: 200 entries
All Codes: 469 entries
Identified code columns: ['code_1', 'code_2', 'code_3', 'code_4']
Column 'code_1': Found 1773 HCPCS matches, 940 Lab matches, 770 CPT matches.
Reassigning code_1 to 'code' and 'code_1_type' to 'type' for non-empty matches DataFrames.
Column 'code_1': Combined matches shape: (3483, 28)
Column 'code_1': Unique matches after dropping duplicates: 3154
Column 'code_2': Found

In [ ]:
# test for matches
codes = ['code_1', 'code_2', 'code_3', 'code_4']

for col in codes:
    print(f"Checking matches for column: {col}")
    hcpcs_matches = wake_med_long_df[wake_med_long_df[col].isin(hcpcs_codes['HCPCS Code'])]
    cpt_matches = wake_med_long_df[wake_med_long_df[col].isin(cpt_codes['HCPCS Code'])]
    lab_matches = wake_med_long_df[wake_med_long_df[col].isin(lab_codes['HCPCS Code'])]
    print(f"  HCPCS matches: {len(hcpcs_matches)}")
    print(f"  CPT matches: {len(cpt_matches)}")
    print(f"  Lab matches: {len(lab_matches)}")

Checking matches for column: code_1
  HCPCS matches: 1773
  CPT matches: 770
  Lab matches: 940
Checking matches for column: code_2
  HCPCS matches: 9384
  CPT matches: 0
  Lab matches: 3248
Checking matches for column: code_3
  HCPCS matches: 0
  CPT matches: 32
  Lab matches: 0
Checking matches for column: code_4
  HCPCS matches: 0
  CPT matches: 0
  Lab matches: 0


In [ ]:
wake_med_long_df['estimated_amount'].notna().sum()

39044

***
## South Carolina Hospitals

**OH BOY AM I PISSED ALREADY**

#### What’s Happening

* **APC codes (`code|1` with type = APC):**
  These rows have payer names, plan names, and estimated amounts. That’s why they’re the only rows showing price info. APC = Ambulatory Payment Classification, a CMS grouping for outpatient procedures.

* **HCPCS/CPT codes (in `code|2`, `code|3`, etc. with type = CPT/HCPCS):**
  These rows often have no payer, plan, or estimated amount attached. Instead, they are mapped *into* the APC buckets, which then carry the pricing/plan info.

* In other words: the hospital publishes payer-specific negotiated rates only at the APC level, while keeping CPT/HCPCS rows as “mappings” without dollar amounts.

#### How We need to Work Around It

1. **Build a crosswalk (mapping):**

   * Use the `description` and `code|n` columns to connect CPT/HCPCS rows to their parent APC row (same description or grouping).
   * Then join those CPT/HCPCS codes to the APC rows that actually carry pricing.
     → This gives you a lookup where searching by CPT/HCPCS leads you to the APC (and thus the estimated amounts and plan names).

2. **Validate mapping:**

   * In practice, hospitals often list the CPT/HCPCS that roll up into each APC.
   * You’ll need to check whether identical descriptions (e.g., “Inj, aflibercept hd, 1 mg”) appear across APC-coded and CPT-coded rows, and merge them.

3. **Practical solution in analysis:**

   * Search by CPT -> Find matching description -> Get its APC -> Pull plan names and estimated amounts from that APC row.

#### On Legality


  Not necessarily illegal. CMS’s **price transparency rule (2021–)** requires hospitals to publish:

  * Gross charges
  * Discounted cash prices
  * Payer-specific negotiated charges
  * De-identified min/max negotiated charges
  * For at least 300 shoppable services (including CPT/HCPCS).

Many hospitals comply only at the APC level (grouping multiple CPTs). This practice has been criticized as undermining the intent of transparency, but hospitals often argue it’s compliant because APCs are “billing codes.” Enforcement has been light, though CMS has fined some hospitals for noncompliance.

## Addendums

Link to addendums for crosswalking https://www.cms.gov/medicare/payment/prospective-payment-systems/hospital-outpatient-pps/quarterly-addenda-updates

In [ ]:
# ======================================================================
# --------------- MUSC HEALTH  ---------------- ADDENDUM B NEEDED
# ======================================================================

import re
from scripts.cleaners import standardize_payer_name
from scripts.cleaners import apply_payer_standardization_to_json
from scripts.bundle_validation import ValidateJSON
from scripts.merge_cpt_to_apc import map_prices_to_hcpcs, load_addendum_b

# Grab row for MUSC Health in Charleston, SC
hos_name = 'MUSC Health'
matching_hospitals = hospitals_df[hospitals_df['name'] == hos_name]
if not matching_hospitals.empty:
    musc_row = matching_hospitals.iloc[0]
else:
    print(f"Hospital '{hos_name}' not found in the dataset")
    musc_row = None

# grab json path for MUSC Health
musc_json_path = musc_row['json_path']

# load a single csv file from csv_folder for testing
musc_csv_path = os.path.join(csv_folder, 'MUSC_Health_Medical_Center_CM.csv')
musc_df = pd.read_csv(musc_csv_path)

# Change all mixed type columns to string to avoid dtype issues (focus codes columns only)
code_cols = [c for c in musc_df.columns if c.startswith("code|") or c.startswith("code_")]
musc_df[code_cols] = musc_df[code_cols].astype(str)

# ======================================================
# ADDENDUM B LOADING
# ======================================================

# load addendum b for mapping, save folder as csv files are stored locally outside of CLEAR repo
addendum_b_path = os.path.join(csv_folder, '2025_Web_Addendum_B.csv')

# load addendum b
addendum_b = load_addendum_b(addendum_b_path)

# Map prices to hcpcs codes in musc_df
musc_df = map_prices_to_hcpcs(musc_df, addendum_b, expand=True)

# replace column name instances with | to _, code|1 becomes code_1, etc
musc_df.columns = [re.sub(r'\|', '_', col) for col in musc_df.columns]

# Code structure is similar to Duke with there being multiple code columns
"""
Checking matches for column: code_1
  HCPCS matches: 0
  CPT matches: 0
  Lab matches: 0
Checking matches for column: code_2
  HCPCS matches: 0
  CPT matches: 10
  Lab matches: 0
Checking matches for column: code_3
  HCPCS matches: 8
  CPT matches: 0
  Lab matches: 323
Checking matches for column: code_4
  HCPCS matches: 0
  CPT matches: 0
  Lab matches: 0

Unique values in code_1_type: ['APC' 'CDM' 'MS-DRG' 'NDC']
Unique values in code_2_type: ['nan' 'RC']
Unique values in code_3_type: ['nan' 'HCPCS']
Unique values in code_4_type: ['nan' 'NDC']

"""

# now grab matches
hcpcs_matches = musc_df[musc_df['code_3'].isin(hcpcs_codes['HCPCS Code'])]
cpt_matches = musc_df[musc_df['code_2'].isin(cpt_codes['HCPCS Code'])]
lab_matches = musc_df[musc_df['code_3'].isin(lab_codes['HCPCS Code'])]

match_dfs = [df for df in [hcpcs_matches, cpt_matches, lab_matches] if not df.empty]  

all_matches = pd.concat(match_dfs, ignore_index=True)

# drop all rows where payer_name is null/empty, for now dont worry about est. price, drop duplicates after
all_matches = all_matches[(all_matches['payer_name'].notna() & (all_matches['payer_name'] != ''))]
all_matches = all_matches.drop_duplicates()

# due to the mapping code types are a bit trickier now, espeically to keep records unique, for 
# now lets just use code_3/code_3_type if present, else code_2/code_2_type for code/type columns
def select_code(row):
    if pd.notna(row['code_3']) and row['code_3'] != '':
        return pd.Series({'code': row['code_3'], 'type': row['code_3_type']})
    elif pd.notna(row['code_2']) and row['code_2'] != '':
        return pd.Series({'code': row['code_2'], 'type': row['code_2_type']})
    else:
        return pd.Series({'code': None, 'type': None})
    
if not all_matches.empty:
    all_matches[['code', 'type']] = all_matches.apply(select_code, axis=1)
    all_matches = all_matches.drop(columns=['code_2', 'code_2_type', 'code_3', 'code_3_type'])

# now lets apply payer standardization
all_matches['payer_name'] = all_matches['payer_name'].apply(standardize_payer_name)

# Now lets remove enteries that don't have est. prices
all_matches = all_matches[all_matches['estimated_amount'].notna() & (all_matches['estimated_amount'] != '')]

# finally export to json
all_matches.to_json(musc_json_path, orient='records', lines=True)

# Validate the final json
validator = ValidateJSON(musc_json_path)

# drop file/df from memory to save space
del musc_df
del musc_row
del musc_csv_path
del all_matches

print("MUSC Health test processing complete.")

# ======================================================================

Matched 1711 / 2015 rows.
service_config.json UPDATED (1 bundle(s) changed). Backup saved.
MUSC Health test processing complete.


***
## Virgina

This will be the firs time tesing the all in one code book, where we crosswalked with the RVU to get info about CPT codes then updated their description and compiled into a singular doc. Here is the output from testing:

#### Original Method:
```python
# column 1 matches
hcpcs_matches = inova_transformed[inova_transformed['code_1'].isin(hcpcs_codes['HCPCS Code'])]
cpt_matches = inova_transformed[inova_transformed['code_1'].isin(cpt_codes['HCPCS Code'])]
lab_matches = inova_transformed[inova_transformed['code_1'].isin(lab_codes['HCPCS Code'])]

# column 2 matches
hcpcs_matches_2 = inova_transformed[inova_transformed['code_2'].isin(hcpcs_codes['HCPCS Code'])]
cpt_matches_2 = inova_transformed[inova_transformed['code_2'].isin(cpt_codes['HCPCS Code'])]
lab_matches_2 = inova_transformed[inova_transformed['code_2'].isin(lab_codes['HCPCS Code'])]

# print out match counts for each
print(f"Column code_1: HCPCS matches: {len(hcpcs_matches)}, CPT matches: {len(cpt_matches)}, Lab matches: {len(lab_matches)}")
print(f"Column code_2: HCPCS matches: {len(hcpcs_matches_2)}, CPT matches: {len(cpt_matches_2)}, Lab matches: {len(lab_matches_2)}")
```
- Column code_1: HCPCS matches: 0, CPT matches: 156, Lab matches: 0
- Column code_2: HCPCS matches: 8054, CPT matches: 4275, Lab matches: 327

#### New Spreadsheet Method:

```python
# column 1 matches
hcpcs_filtered_codes = all_codes[all_codes['source'] == 'HCPCS']
cpt_filtered_codes = all_codes[all_codes['source'] == 'CPT']
lab_filtered_codes = all_codes[all_codes['source'] == 'Lab']

# column 1 matches
hcpcs_matches = inova_transformed[inova_transformed['code_1'].isin(hcpcs_filtered_codes['code'])]
cpt_matches = inova_transformed[inova_transformed['code_1'].isin(cpt_filtered_codes['code'])]
lab_matches = inova_transformed[inova_transformed['code_1'].isin(lab_filtered_codes['code'])]

# column 2 matches
hcpcs_matches_2 = inova_transformed[inova_transformed['code_2'].isin(hcpcs_filtered_codes['code'])]
cpt_matches_2 = inova_transformed[inova_transformed['code_2'].isin(cpt_filtered_codes['code'])]
lab_matches_2 = inova_transformed[inova_transformed['code_2'].isin(lab_filtered_codes['code'])]

# print out match counts for each
print(f"Column code_1: HCPCS matches: {len(hcpcs_matches)}, CPT matches: {len(cpt_matches)}, Lab matches: {len(lab_matches)}")
print(f"Column code_2: HCPCS matches: {len(hcpcs_matches_2)}, CPT matches: {len(cpt_matches_2)}, Lab matches: {len(lab_matches_2)}")
```
- Column code_1: HCPCS matches: 0, CPT matches: 0, Lab matches: 0
- Column code_2: HCPCS matches: 8054, CPT matches: 2345, Lab matches: 288

We can see tha the original is catching more CPT matches as well as Lab, but there is a chance that it's catching codes that are depreciated. This needs to be investigated further but I already have enough on my plate.

In [ ]:
# =====================================================================
#  Inova Fairfax Hospital 
# =====================================================================

from scripts.cleaners import transform_wide_to_long_format

hos_name = 'Inova Fairfax Hospital'
address = '3300 Gallows Rd'
city_name = 'Falls Church'
state_name = 'VA'
zip_code = '22042'

# update hospitals_df with new entry if it doesn't already exist
# move this to be a function later
if hospitals_df[
    (hospitals_df['name'] == hos_name) &
    (hospitals_df['state'] == state_name) &
    (hospitals_df['city'] == city_name)
].empty:
    new_entry = {
        'name': hos_name,
        'address': address,
        'city': city_name,
        'state': state_name,
        'zip': zip_code
    }
    hospitals_df = pd.concat([hospitals_df, pd.DataFrame([new_entry])], ignore_index=True)
    hospitals_df.to_csv(hospitals_csv, index=False)  # Save updated CSV
    print(f"Added new hospital entry for '{hos_name}' to hospitals.csv")

    # now run the update_dataframe function to add lat/lon, id, and json_path
    update_dataframe(hospitals_df)

# now lets grab all the paths
inova_json_path = hospitals_df[hospitals_df['name'] == hos_name]['json_path'].values[0]
inova_csv_path = os.path.join(csv_folder, 'INOVA_FAIRFAX_CM.csv')

# load Inova Fairfax Hospital csv
inova_df = pd.read_csv(inova_csv_path)

# Inova is another wide format CM structure, so we need to transform it
# Apply the transformation
print("Transforming Inova Fairfax data from wide to long format...")
inova_transformed = transform_wide_to_long_format(inova_df, )

# Original shape: (19293, 248)
# Transformed shape: (145308, 16)

# Now to grab matches
# The bulk of the matches are in code_2/code_2_type, so we will just use those for now due to limited space
# HCPCS matches
hcpcs_matches = inova_transformed[inova_transformed['code_2'].isin(hcpcs_codes['HCPCS Code'])].copy()
hcpcs_matches['code'] = hcpcs_matches['code_2']
hcpcs_matches['type'] = hcpcs_matches['code_2_type']
hcpcs_matches = hcpcs_matches.drop(columns=['code_1', 'code_1_type', 'code_2', 'code_2_type'])

# CPT matches
cpt_matches = inova_transformed[inova_transformed['code_1'].isin(cpt_codes['HCPCS Code'])].copy()
cpt_matches['code'] = cpt_matches['code_2']
cpt_matches['type'] = cpt_matches['code_2_type']
cpt_matches = cpt_matches.drop(columns=['code_1', 'code_1_type', 'code_2', 'code_2_type'])

# Lab matches
lab_matches = inova_transformed[inova_transformed['code_2'].isin(lab_codes['HCPCS Code'])].copy()
lab_matches['code'] = lab_matches['code_2']
lab_matches['type'] = lab_matches['code_2_type']
lab_matches = lab_matches.drop(columns=['code_1', 'code_1_type', 'code_2', 'code_2_type'])

# Combine all matches into one dataframe, drop duplicates
match_dfs = [df for df in [hcpcs_matches, cpt_matches, lab_matches] if not df.empty]
if match_dfs:
    all_matches = pd.concat(match_dfs, ignore_index=True).drop_duplicates()
    all_matches['payer_name'] = all_matches['payer_name'].apply(standardize_payer_name)

    # redrop possible duplicates
    all_matches = all_matches.drop_duplicates()

# now lets drop some more columns to save space
cols_to_drop = ['billing_class', 'drug_unit_of_measurement', 'drug_type_of_measurement', 
                'modifiers', 'standard_charge_algorithm', 'additional_generic_notes', 'methodology']
all_matches = all_matches.drop(columns=cols_to_drop, errors='ignore')

# Now we can save to json
all_matches.to_json(inova_json_path, orient='records', lines=True)

# drop file/df from memory to save space
del inova_df
del inova_transformed
del all_matches
del inova_csv_path
del inova_json_path

print("Inova Fairfax Hospital test processing complete.")

Transforming Inova Fairfax data from wide to long format...
Inova Fairfax Hospital test processing complete.
Inova Fairfax Hospital test processing complete.


***
## Medicare Pricing Integration

Testing the new enhanced pricing reader that can automatically parse CMS pricing files and match codes from our top 200 lists to Medicare rates. This includes ASC, ASP, CLFS, DMEPOS, PFALL pricing sources plus calculated anesthesia rates.

In [ ]:
# Test the enhanced pricing reader functionality
import importlib
import sys

# Reload the module to pick up changes
if 'scripts.enhanced_pricing_reader' in sys.modules:
    importlib.reload(sys.modules['scripts.enhanced_pricing_reader'])

from scripts.enhanced_pricing_reader import match_codes_to_pricing, test_pricing_reader

# First, let's test the individual pricing file readers
print("Testing pricing file readers...")
pricing_folder_path = os.path.join(workspace_root, '..', 'ChargeMaster_Project', 'pricing_info')
pricing_folder_path = os.path.abspath(pricing_folder_path)

# Run the test function
test_pricing_reader(pricing_folder_path)

Testing pricing file readers...
Testing individual parsers...
ASC test: 4723 records loaded
  Sample: 0101T = $240.17
ASP test: 871 records loaded
  Sample: 90653 = $83.49
CLFS test: 1926 records loaded
  Sample: 0001U = $720.0
DMEPOS test: 2059 records loaded
  Sample: A4216 = $0.97
PFALL test: 7481 records loaded
  Sample: G0011 = $31.56
Anesthesia test: 5 base units + 68 minutes = $208.62
PFALL test: 7481 records loaded
  Sample: G0011 = $31.56
Anesthesia test: 5 base units + 68 minutes = $208.62


In [ ]:
# Now let's match our loaded codes to Medicare pricing
print("\n" + "="*60)
print("MATCHING CODES TO MEDICARE PRICING")
print("="*60)

# Prepare the list of code dataframes
code_dataframes = [hcpcs_codes, lab_codes, cpt_codes, all_codes]

# Match codes to pricing and create unified output
output_path = os.path.join(workspace_root, 'docs', 'data', 'medicare_pricing_matched.csv')
matched_pricing = match_codes_to_pricing(
    code_dataframes=code_dataframes,
    pricing_folder=pricing_folder_path,
    output_file=output_path,
    include_anesthesia=True
)

print(f"\nMatched pricing data shape: {matched_pricing.shape}")
if not matched_pricing.empty:
    print(f"Price range: ${matched_pricing['price'].min():.2f} - ${matched_pricing['price'].max():.2f}")
    print(f"Average price: ${matched_pricing['price'].mean():.2f}")
    
    # Show sample of results
    print(f"\nSample matched results:")
    print(matched_pricing.head(10)[['code', 'price', 'source']].to_string(index=False))


MATCHING CODES TO MEDICARE PRICING
Starting code matching process...
Parsing all pricing files...
ASC: 4723 records
ASP: 871 records
CLFS: 1926 records
DMEPOS: 2059 records
PFALL: 7481 records
Total unique codes to match: 478
PFALL: 7481 records
Total unique codes to match: 478

Matching Results:
Total codes processed: 478
Successfully matched: 450
Unmatched codes: 28
Match rate: 94.1%

Pricing sources used:
  PFALL: 189 codes
  ASC: 96 codes
  CLFS: 79 codes
  ASP: 48 codes
  DMEPOS: 38 codes

Results saved to: c:\Users\jcing\OneDrive\Desktop\MADS\DATA 760\Project\CLEAR\docs\data\medicare_pricing_matched.csv
Unmatched codes saved to: c:\Users\jcing\OneDrive\Desktop\MADS\DATA 760\Project\CLEAR\docs\data\medicare_pricing_matched_unmatched.csv

Matched pricing data shape: (450, 4)
Price range: $0.01 - $14070.52
Average price: $457.20

Sample matched results:
 code   price source
87801  70.200   CLFS
Q5101   0.360    ASC
J0461   0.107    ASP
J7209   1.240    ASC
A6216   0.090 DMEPOS
A506